In [2]:
import pandas as pd
import requests
import numpy as np

ModuleNotFoundError: No module named 'requests'

In [48]:
data = pd.read_csv("C:/Users/arneg/Desktop/Google ML Immersion/URL PROJECT/SSH Public/HiddenFraudulentURLs.csv",delimiter=";")

______________________________________________________________________________________________

<b>1) DUPLICATES SERVE NO PURPOSE.  EVEN THOUGH CONTENT LENGTH CHANGES, NO DIFFERENCES</b>

In [49]:
df_rd = data.drop_duplicates()

In [50]:
df_rd['url2'] = df_rd['url']

C:\Users\arneg\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
test = df_rd.groupby(['url','isHiddenFraudulent']).url2.count().reset_index().sort_values('url2',ascending=False)

In [ ]:
test[(test['url2'] > 1) & (test['isHiddenFraudulent'] == True)]

In [ ]:
df_rd[df_rd['url'] == "http://www.kinnaree.co.uk/x.txt"]

______________________________________________________________________________________________


<b>2) CONTENT LENGTH SEEMS TO HAVE SIGNIFICANCE</b>

In [ ]:
df_rd.groupby(['compromissionType','isHiddenFraudulent']).contentLength.mean().reset_index()

In [ ]:
df_rd.groupby(['compromissionType','isHiddenFraudulent']).contentLength.describe()

______________________________________________________________________________________________


<b>3) Powered By That Causes the Most Shenanigans</b>

In [ ]:
powered = df_rd.groupby(['poweredBy','isHiddenFraudulent']).url.count().reset_index().sort_values(['isHiddenFraudulent','url'])

In [ ]:
powered2 = pd.DataFrame(powered.pivot_table(index='poweredBy',
                                           values='url',
                                           columns='isHiddenFraudulent'))
powered2.replaceNAs

In [ ]:
df_server = df_rd[df_rd['serverType'].isna()]

In [ ]:
df_server['serverType_New'] = ""
df_server['serverType_New'] = df_server.url.apply(lambda row: getServer(row))
#df_server['serverType_New'] = ""
#df_server.reset_index(inplace=True)

In [ ]:
def getServer(url):
    try:
        temp = requests.get(url)
        temp = temp.headers['server']
    except:
        temp = "NA"
    return temp

In [ ]:
df_server.groupby(['serverType','isHiddenFraudulent']).url.count().reset_index()

In [ ]:
df_server['serverType'] = df_server['serverType_New']
df_server.drop('serverType_New',axis=1,inplace=True)

In [ ]:
x = df_rd[df_rd['serverType'].isna() == False].drop('serverType',axis=1)
x = pd.concat([x,df_server],axis=0)
x

In [ ]:
df_rd2 = df_rd[df_rd['serverType'].isna() == False]
df_rd2 = df_rd2 + pd.concat(df_rd[df_rd['serverType'].isna() == False].drop('serverType',axis=1),df_server['serverType'],axis=1)

In [ ]:
pip install python-whois

# HAS IP COLUMN CODE

In [94]:
import re

In [151]:
pat = re.compile("((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)")
    #"((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)")
    #"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")
def ipCheck(x):
    x = re.sub(r'[^0-9.]', '', x)
    test = pat.match(x)
    if test:
        return 1
    else:
        return 0

In [152]:
df_rd2 = df_rd

In [153]:
df_rd2['hasIP'] = df_rd2.url.apply(lambda row: ipCheck(row))

C:\Users\arneg\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [155]:
df_rd2[df_rd2['hasIP']==1].count()

url                   49
compromissionType     49
isHiddenFraudulent    49
contentLength         49
serverType            49
poweredBy             45
contentType           49
lastModified          37
url2                  49
hasIP                 49
dtype: int64

# PRIMARY DOMAIN

In [6]:
import urllib.parse as urlparse
from os.path import splitext, basename

In [17]:
def primaryDomain(x,l='len'):
    temp = urlparse.urlparse(x).hostname
    if l == 'len':
        return len(temp)
    else:
        return temp

In [18]:
t = primaryDomain("http://aaa.bjgdhm.com/show_products.asp",l='str')
t

'aaa.bjgdhm.com'

# PARSE URL CHUNKS

In [11]:
validExtensions = pd.read_excel("./Extensions.xlsx")

In [12]:
validExtensions = list(validExtensions['Ext'])

In [21]:
def extFind(url, combine=True):
    t = primaryDomain(url,l='str')
    orig = t
    t = t.split(".")
    temp = ""
    last = t[-1]
    while (len(last) <= 3) & (last in validExtensions):
        if temp == "":
            temp = t.pop()
        else:
            temp = t.pop() + "." + temp
        try:
            last = t[-1]
        except IndexError:
            break
    if combine == True:
        if orig[:2] =="ww":
            t = [t[0],".".join(t[1:])]
        else:
            t = [".".join(t)]
    t.append(temp)
    #print(str(orig) + " --> " + str(t))
    return t

In [1]:
from helpers_proc import primaryDomain, extFind

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("./test.csv", sep=',')

In [10]:
df['domain_temp'] = ""
df["domain_temp"] = df.url.apply(lambda row: extFind(row,True))
def urlVars(df, domain_list_col):
    df['URL_Length'] = df.url.apply(lambda row: len(row))
    df['Query'] = df.url.apply(lambda row: urlparse.urlparse(row).query if urlparse.urlparse(row).query != '' else 'None')
    df['Query_Length'] = df.Query.apply(lambda row: len(row))
    df['Primary'] = df[domain_list_col].apply(lambda row: row[1] if row[0] in ['www','ww1','ww2'] else row[0])
    df['Primary_Length'] = df.Primary.apply(lambda row: len(row))
    df['Extension'] = df[domain_list_col].apply(lambda row: row[-1])
    df['Extension_Length'] = df['Extension'].apply(lambda row: len(row))
    df['Num_Periods'] = df.url.apply(lambda row: row.count("."))
    df['Num_Exclam'] = df.url.apply(lambda row: row.count("!"))
    df['Num_Quest'] = df.url.apply(lambda row: row.count("?"))
    df['Num_Perc'] = df.url.apply(lambda row: row.count("%"))
    df['Num_Hyphen'] = df.url.apply(lambda row: row.count("-"))
    df['Num_Numbers'] = df.url.apply(lambda row: sum(c.isdigit() for c in row))
    df['Words'] = df.url.apply(lambda row: "".join((char if char.isalnum() else " ") for char in row).split())
    df['Word_Counts'] = df.Words.apply(lambda row: [len(x) for x in row])
    df['Longest_Word'] = df.Word_Counts.apply(lambda row: max(row))
    df['Shortest_Word'] = df.Word_Counts.apply(lambda row: min(row))
    df['Mean_Word'] = df.Word_Counts.apply(lambda row: float(sum(row))/max(len(row),1))

In [7]:
urlVars(df, 'domain_temp')

In [9]:
df

,url,compromissionType,isHiddenFraudulent,contentLength,serverType,poweredBy,contentType,lastModified,domain_temp,URL_Length,...,Num_Exclam,Num_Quest,Num_Perc,Num_Hyphen,Num_Numbers,Words,Word_Counts,Longest_Word,Shortest_Word,Mean_Word
0,http://www.sinduscongoias.com.br/index.php/fal...,defacement,False,0,Apache/2.2,Unknown,text/html; charset=utf-8,"Mon, 21 Jan 2013 19:31:19 GMT","[www, sinduscongoias, com.br]",55,...,0,0,0,1,0,"[http, www, sinduscongoias, com, br, index, ph...","[4, 3, 14, 3, 2, 5, 3, 4, 7]",14,2,5.000000
1,http://www.pontoprofissional.com.br/portal/ind...,defacement,False,0,Microsoft-IIS/6.0,Unknown,text/html; charset=utf-8,"Tue, 22 Jan 2013 00:28:49 GMT","[www, pontoprofissional, com.br]",100,...,0,1,0,0,4,"[http, www, pontoprofissional, com, br, portal...","[4, 3, 17, 3, 2, 6, 5, 3, 6, 3, 7, 4, 7, 2, 2,...",17,2,4.823529
2,http://www.pontoprofissional.com.br/portal/ind...,defacement,False,0,Microsoft-IIS/6.0,Unknown,text/html; charset=utf-8,"Tue, 22 Jan 2013 00:28:56 GMT","[www, pontoprofissional, com.br]",100,...,0,1,0,0,4,"[http, www, pontoprofissional, com, br, portal...","[4, 3, 17, 3, 2, 6, 5, 3, 6, 3, 7, 4, 7, 2, 2,...",17,2,4.823529
3,http://www.coleyglesias.com/index.html,defacement,False,0,Apache,Unknown,text/html; charset=utf-8,"Tue, 22 Jan 2013 02:30:56 GMT","[www, coleyglesias, com]",38,...,0,0,0,0,0,"[http, www, coleyglesias, com, index, html]","[4, 3, 12, 3, 5, 4]",12,3,5.166667
4,http://www.coleyglesias.com/index.php?option=c...,defacement,False,0,Apache,Unknown,text/html; charset=utf-8,"Tue, 22 Jan 2013 02:30:59 GMT","[www, coleyglesias, com]",98,...,0,1,0,0,4,"[http, www, coleyglesias, com, index, php, opt...","[4, 3, 12, 3, 5, 3, 6, 3, 7, 4, 8, 6, 4, 2, 2,...",12,2,4.705882
5,http://www.coleyglesias.com/index.php?option=c...,defacement,False,0,Apache,Unknown,text/html; charset=utf-8,"Tue, 22 Jan 2013 02:31:03 GMT","[www, coleyglesias, com]",98,...,0,1,0,0,4,"[http, www, coleyglesias, com, index, php, opt...","[4, 3, 12, 3, 5, 3, 6, 3, 7, 4, 8, 6, 4, 2, 2,...",12,2,4.705882
6,http://www.coleyglesias.com/index.php?option=c...,defacement,False,0,Apache,Unknown,text/html; charset=utf-8,"Tue, 22 Jan 2013 02:31:04 GMT","[www, coleyglesias, com]",98,...,0,1,0,0,4,"[http, www, coleyglesias, com, index, php, opt...","[4, 3, 12, 3, 5, 3, 6, 3, 7, 4, 8, 6, 4, 2, 2,...",12,2,4.705882
7,http://www.coleyglesias.com/index.php?option=c...,defacement,False,0,Apache,Unknown,text/html; charset=utf-8,"Tue, 22 Jan 2013 02:31:09 GMT","[www, coleyglesias, com]",98,...,0,1,0,0,4,"[http, www, coleyglesias, com, index, php, opt...","[4, 3, 12, 3, 5, 3, 6, 3, 7, 4, 8, 6, 4, 2, 2,...",12,2,4.705882
8,http://www.coleyglesias.com/index.php?option=c...,defacement,False,0,Apache,Unknown,text/html; charset=utf-8,"Tue, 22 Jan 2013 02:31:22 GMT","[www, coleyglesias, com]",176,...,0,1,0,0,16,"[http, www, coleyglesias, com, index, php, opt...","[4, 3, 12, 3, 5, 3, 6, 3, 6, 4, 9, 4, 100]",100,3,12.461538
9,http://www.coleyglesias.com/index.php?option=c...,defacement,False,0,Apache,Unknown,text/html; charset=utf-8,"Tue, 22 Jan 2013 02:31:27 GMT","[www, coleyglesias, com]",138,...,0,1,0,4,6,"[http, www, coleyglesias, com, index, php, opt...","[4, 3, 12, 3, 5, 3, 6, 3, 7, 4, 7, 2, 3, 8, 1,...",12,1,4.956522


In [32]:
df.loc[df['Query'].str.contains('tag=')].Query

3405                   option=com_tag&task=tag&tag=vortrag
3406                      option=com_tag&task=tag&tag=Sylt
3407     option=com_tag&task=tag&tag=ferienhaus+mit+sau...
3408             option=com_tag&task=tag&tag=veranstaltung
4325                   option=com_tag&task=tag&tag=problem
4326                      option=com_tag&task=tag&tag=tips
5997                                           tag=rumfart
5998                                             tag=video
6006                                               tag=jul
6008                                         tag=videnskab
6011                                         tag=solsystem
6012                                         tag=asteroide
6013                                   feed=rss2&tag=video
6017                                        tag=opfindelse
6018                                     tag=video&paged=2
6019                                               tag=fly
6020                                          tag=luftfa

In [536]:
for word in df_rd2['Words']:
    for x in word:
        if (x == "") | (x == None):
            print(word)